In [1]:
# 数据 load

import glob
import shutil

valid_dir = "output/hf-eval-data-v3-valid/"

def load_valid_data(dir_path):
    data = []
    file_pattern = "*.py"

    # 使用 glob.glob 获取匹配的文件列表
    python_files = glob.glob(dir_path + file_pattern)

    for file_path in python_files:
        try:
            with open(file_path, 'r') as file:
                content = file.read()
                data.append({
                    "path": file_path,
                    "content": content,
                })
                print(f"内容来自文件: {file_path}\n")
        except FileNotFoundError:
            print(f"文件 {file_path} 未找到。")
        except Exception as e:
            print(f"发生错误：{e}")
            
    return data

valid_data = load_valid_data(valid_dir)
valid_data[0], len(valid_data)

内容来自文件: output/hf-eval-data-v3-valid/f00002_extract_medical_term_relationships.py

内容来自文件: output/hf-eval-data-v3-valid/f00004_extract_sentence_embeddings.py

内容来自文件: output/hf-eval-data-v3-valid/f00016_load_graphormer_model.py

内容来自文件: output/hf-eval-data-v3-valid/f00018_estimate_image_depth.py

内容来自文件: output/hf-eval-data-v3-valid/f00020_classify_image.py

内容来自文件: output/hf-eval-data-v3-valid/f00035_sentiment_analysis.py

内容来自文件: output/hf-eval-data-v3-valid/f00049_get_legal_answer.py

内容来自文件: output/hf-eval-data-v3-valid/f00051_get_answer_from_document.py

内容来自文件: output/hf-eval-data-v3-valid/f00053_news_category_detection.py

内容来自文件: output/hf-eval-data-v3-valid/f00058_summarize_text.py

内容来自文件: output/hf-eval-data-v3-valid/f00059_summarize_text.py

内容来自文件: output/hf-eval-data-v3-valid/f00060_summarize_text.py

内容来自文件: output/hf-eval-data-v3-valid/f00069_compute_sentence_embeddings.py

内容来自文件: output/hf-eval-data-v3-valid/f00071_get_sentence_embedding.py

内容来自文件: output/hf-eval-dat

({'path': 'output/hf-eval-data-v3-valid/f00002_extract_medical_term_relationships.py',
  'content': '# function_import --------------------\n\nfrom transformers import AutoTokenizer, AutoModel\nimport torch\n\n# function_code --------------------\n\ndef extract_medical_term_relationships(medical_term):\n    """\n    This function uses the pretrained model \'GanjinZero/UMLSBert_ENG\' from Hugging Face Transformers to find relationships between medical terms.\n    It converts the medical terms into embeddings (dense vectors) which can be compared to find similarities and relationships.\n\n    Args:\n        medical_term (str): The medical term to be converted into an embedding.\n\n    Returns:\n        torch.Tensor: The embedding of the input medical term.\n    """\n    tokenizer = AutoTokenizer.from_pretrained(\'GanjinZero/UMLSBert_ENG\')\n    model = AutoModel.from_pretrained(\'GanjinZero/UMLSBert_ENG\')\n\n    inputs = tokenizer(medical_term, return_tensors=\'pt\')\n    outputs = mode

In [2]:
# 拆出 instruct / output / testing
import re

def extract_between_strings(input_string, start_string, end_string):
    after = input_string.split(start_string)[1]
    if end_string != "":
        between = after.split(end_string)[0]
        return start_string + between
    
    return start_string + after

def extract_section(data):
    content = data['content']
    
    s = "# function_import --------------------"
    e = "# function_code --------------------"
    function_import = extract_between_strings(content, s, e)
    
    s = "# function_code --------------------"
    e = "# test_function_code --------------------"
    function_code = extract_between_strings(content, s, e)
    
    s = "# test_function_code --------------------"
    e = "# call_test_function_code --------------------"
    test_function_code = extract_between_strings(content, s, e)
    
    s = "# call_test_function_code --------------------"
    e = ""
    call_test_function_code = extract_between_strings(content, s, e)
    
    data['function_import'] = function_import
    data['function_code'] = function_code
    data['test_function_code'] = test_function_code
    data['call_test_function_code'] = call_test_function_code
    
    return data

result = extract_section(valid_data[89])
result

{'path': 'output/hf-eval-data-v3-valid/f00385_identify_company_names.py',
 'content': '# function_import --------------------\n\nimport torch\nfrom transformers import AutoModelForTokenClassification, AutoTokenizer\n\n# function_code --------------------\n\ndef identify_company_names(text):\n    """\n    Identify company names from a given text using a pre-trained model from Hugging Face Transformers.\n\n    Args:\n        text (str): The input text from which company names are to be identified.\n\n    Returns:\n        outputs (torch.Tensor): The model outputs, which include the predicted token classifications.\n\n    Raises:\n        ValueError: If the input text is not a string.\n    """\n    if not isinstance(text, str):\n        raise ValueError(\'Input text must be a string.\')\n\n    model = AutoModelForTokenClassification.from_pretrained(\'ismail-lucifer011/autotrain-company_all-903429548\')\n    tokenizer = AutoTokenizer.from_pretrained(\'ismail-lucifer011/autotrain-company_al

In [7]:
# 拆出注释、正文
def extract_all_section(data):
    result = extract_section(data)
        
    s = "# function_code --------------------"
    e = "'''"
    if e not in result['function_code']:
        e = '"""'

    def extract_instruct(input_string, start_string, end_string):
        after = input_string.split(start_string)[1]
        between = after.rsplit(end_string, 1)[0]
        return start_string + between + end_string

    def extract_answer(input_string, start_string):
        after = input_string.split(start_string)[-1]
        return after

    instruct = extract_instruct(result['function_code'], s, e)
    answer = extract_answer(result['function_code'], e)
    
    result['instruct'] = result['function_import'] + instruct
    result['answer'] = answer
    
    return result


import traceback

section_valid_data = []
for d in valid_data:
    try:
        result = extract_all_section(d)
        section_valid_data.append(result)
    except Exception as e:
        # Handle the exception and print the traceback
        print("An exception occurred:", e)
        traceback.print_exc()
    
len(section_valid_data), section_valid_data[0]

(241,
 {'path': 'output/hf-eval-data-v3-valid/f00002_extract_medical_term_relationships.py',
  'content': '# function_import --------------------\n\nfrom transformers import AutoTokenizer, AutoModel\nimport torch\n\n# function_code --------------------\n\ndef extract_medical_term_relationships(medical_term):\n    """\n    This function uses the pretrained model \'GanjinZero/UMLSBert_ENG\' from Hugging Face Transformers to find relationships between medical terms.\n    It converts the medical terms into embeddings (dense vectors) which can be compared to find similarities and relationships.\n\n    Args:\n        medical_term (str): The medical term to be converted into an embedding.\n\n    Returns:\n        torch.Tensor: The embedding of the input medical term.\n    """\n    tokenizer = AutoTokenizer.from_pretrained(\'GanjinZero/UMLSBert_ENG\')\n    model = AutoModel.from_pretrained(\'GanjinZero/UMLSBert_ENG\')\n\n    inputs = tokenizer(medical_term, return_tensors=\'pt\')\n    outputs 

In [18]:
# 输出到文件
import json

with open(f"output/hf-eval-data-v3-instruct.jsonl", 'w') as f:
    for d in section_valid_data:
        f.write(json.dumps(d) + "\n")

In [19]:
import pprint
pprint.pp(section_valid_data[0])

{'path': 'output/hf-eval-data-v3-valid/f00002_extract_medical_term_relationships.py',
 'content': '# function_import --------------------\n'
            '\n'
            'from transformers import AutoTokenizer, AutoModel\n'
            'import torch\n'
            '\n'
            '# function_code --------------------\n'
            '\n'
            'def extract_medical_term_relationships(medical_term):\n'
            '    """\n'
            '    This function uses the pretrained model '
            "'GanjinZero/UMLSBert_ENG' from Hugging Face Transformers to find "
            'relationships between medical terms.\n'
            '    It converts the medical terms into embeddings (dense vectors) '
            'which can be compared to find similarities and relationships.\n'
            '\n'
            '    Args:\n'
            '        medical_term (str): The medical term to be converted into '
            'an embedding.\n'
            '\n'
            '    Returns:\n'
            '  

In [67]:
# ! cp output/hf-eval-v3-240.json /root/autodl-tmp/LLaMA-Factory/data/

In [69]:
# import json

# d = {}

# with open("/root/autodl-tmp/LLaMA-Factory/data/dataset_info.json") as f:
#     d = json.loads(f.read())
#     d['hf_eval_v3_240'] = {
#         "file_name": "hf-eval-v3-240.json",
#         "columns": {
#           "prompt": "instruction",
#           "query": "input",
#           "response": "output",
#           "history": "history"
#         }
#     }
    
# with open("/root/autodl-tmp/LLaMA-Factory/data/dataset_info.json", 'w') as f:
#     f.write(json.dumps(d))

In [1]:
%env REPLICATE_API_TOKEN=r8_1YU2oz2exxBYZYIo9sZDngjFwBNvNQD0IiXjD

env: REPLICATE_API_TOKEN=r8_1YU2oz2exxBYZYIo9sZDngjFwBNvNQD0IiXjD


In [3]:
import replicate

def get_prediction(prompt, model="vixuowis/codellama-13b-python"):
    deployment = replicate.deployments.get(model)
    prediction = deployment.predictions.create(
      input={"prompt": prompt}
    )
    prediction.wait()
    return "".join(prediction.output)

In [5]:
import json
import traceback

with open("output/hf-eval-data-v3-instruct.jsonl") as f:
    for idx, l in enumerate(f):
        d = json.loads(l)
        idx_str = str(idx + 1).zfill(5)
        with open(f"output/hf-eval-data-v3-result-13b/result-{idx_str}.json", 'w') as fw:
            try:
                print(idx_str, end="...")
                d['prediction'] = get_prediction(d['instruct'], "vixuowis/codellama-13b-python")
                fw.write(json.dumps(d))
            except Exception as e:
                # Handle the exception and print the traceback
                print("An exception occurred:", e)
                traceback.print_exc()

00001...00002...00003...00004...00005...00006...00007...00008...00009...00010...00011...00012...00013...00014...00015...00016...00017...00018...An exception occurred: Expecting value: line 1 column 1 (char 0)
00019...

Traceback (most recent call last):
  File "/tmp/ipykernel_4874/4280536160.py", line 11, in <cell line: 4>
    d['prediction'] = get_prediction(d['instruct'], "vixuowis/codellama-13b-python")
  File "/tmp/ipykernel_4874/4135852144.py", line 8, in get_prediction
    prediction.wait()
  File "/root/miniconda3/lib/python3.8/site-packages/replicate/prediction.py", line 126, in wait
    self.reload()
  File "/root/miniconda3/lib/python3.8/site-packages/replicate/prediction.py", line 142, in reload
    updated = self._client.predictions.get(self.id)
  File "/root/miniconda3/lib/python3.8/site-packages/replicate/prediction.py", line 240, in get
    resp = self._client._request("GET", f"/v1/predictions/{id}")
  File "/root/miniconda3/lib/python3.8/site-packages/replicate/client.py", line 85, in _request
    _raise_for_status(resp)
  File "/root/miniconda3/lib/python3.8/site-packages/replicate/client.py", line 358, in _raise_for_status
    raise ReplicateError(resp.json()["detail"])
  File "/roo

An exception occurred: Expecting value: line 1 column 1 (char 0)
00020...

Traceback (most recent call last):
  File "/tmp/ipykernel_4874/4280536160.py", line 11, in <cell line: 4>
    d['prediction'] = get_prediction(d['instruct'], "vixuowis/codellama-13b-python")
  File "/tmp/ipykernel_4874/4135852144.py", line 8, in get_prediction
    prediction.wait()
  File "/root/miniconda3/lib/python3.8/site-packages/replicate/prediction.py", line 126, in wait
    self.reload()
  File "/root/miniconda3/lib/python3.8/site-packages/replicate/prediction.py", line 142, in reload
    updated = self._client.predictions.get(self.id)
  File "/root/miniconda3/lib/python3.8/site-packages/replicate/prediction.py", line 240, in get
    resp = self._client._request("GET", f"/v1/predictions/{id}")
  File "/root/miniconda3/lib/python3.8/site-packages/replicate/client.py", line 85, in _request
    _raise_for_status(resp)
  File "/root/miniconda3/lib/python3.8/site-packages/replicate/client.py", line 358, in _raise_for_status
    raise ReplicateError(resp.json()["detail"])
  File "/roo

00021...00022...00023...00024...00025...00026...00027...00028...00029...00030...00031...00032...00033...00034...00035...00036...00037...00038...00039...00040...00041...00042...00043...00044...00045...00046...00047...00048...00049...00050...00051...00052...00053...00054...00055...00056...00057...00058...00059...00060...00061...00062...00063...00064...00065...00066...00067...00068...00069...00070...00071...00072...00073...00074...00075...00076...00077...00078...00079...00080...00081...00082...00083...00084...00085...00086...00087...00088...00089...00090...00091...00092...00093...00094...00095...00096...00097...00098...00099...00100...00101...00102...00103...00104...00105...00106...00107...00108...00109...00110...00111...00112...00113...00114...00115...00116...00117...00118...00119...00120...00121...00122...00123...00124...00125...00126...00127...00128...00129...00130...00131...00132...00133...00134...00135...00136...00137...00138...00139...00140...00141...00142...00143...00144...00145...